# Overview

Tutorial on how to create, connect, fill, request and modify a MongoDB.
This tutorial is part of the DataBases Tutorials Series.

## Plan
1. Create the environment (using UV) to create a MongoDB and a FastAPI API.
2. Create a simple API using FastAPI that can add (POST) a new User and show (GET) a list of Users:
    1. with uvicorn we can run it locally and test it's output even before connecting it to a database.
3. Create a **local MongoDB database** using Docker. Initialize it with a container named `users`.
4. Connect **manually** the API (hosted using Uvicorn) w/ the MongoDB database from 2).
5. Add data into the database and persist it using *volumes*.
6. Repeat using **Docker Compose** to handle the orchestration


## 1 - Environment



Create a folder for the API

In [ ]:
mkdir api

Init an [UV]() project inside the `api` dir:

In [ ]:
cd api && uv init .

Install dependencies to create the API, host:

In [ ]:
uv add fastapi motor uvicorn pydantic

Create the `main.py` file,
inside `api/`,
to execute on the server,
and copy the following code inside it.

```python
from fastapi import FastAPI
api = FastAPI()

```

## 2 - Create Local MongoDB

We will use Docker to create the instance and MongoDB Compose to visualize and create the `collection` and `database`.

In a Terminal run


In [ ]:
docker run -d --name mongodb-demo -p 27017:27017 -v mongodemo:/data/db mongo:latest

Command explained:

1.  This will create a container called `mongodb-demo` with `mongo` (latest version) image.

2.  It will connect machine port `27017` to the container port `27017` (mongo default port),
as expressed by the flag `-p <pc-port>:<container-port>`.

3.  Finally since we want to persist the data even if we destroy de container we create a volume 'mongodemo' using the local directory `.` to store data.

### Initialize DataBase and Collection 


#### with MongoDB Compass

Install [MongoDB Compass]() and connect to it using as connection string:
`mongodb://localhost:27017/`.
Note that this implies no `user` nor `password`, i.e. no authentication;
we will add that later.

Once connected select `Create Database` and call it `testDB`.
Inside it select `Create Collection` and call it `users`.

#### with MongoSh

Once the container for the DB is running,
which we can check with:
```bash
docker ps
```

To connect to the DB using Mongosh copy the `CONTAINER ID` and use it in the following command:
```bash
docker exec -it <CONTAINER-ID> bash &&
mongosh
```

- show existing databases: `show dbs`
- create the new database `test-database`: `use test-database`
- show collections: `show collections`
- create a new collection `users`:
    - by inserting one`db.users.insertOne()`
- show elements inside collection: `db.users.find().pretty()`

>[!important] If a collection does not exist, MongoDB creates the collection when you first store data for that collection.


## 3 - Create an API with FastAPI to interact with the DB

1.  Create everything inside `main.py`
2.  Structure in a pkg way to organize the codebase better.

Inside `api/main.py` add the following lines to define a new API:

```python
import os
from motor.motor_asyncio import AsyncIOMotorClient
from fastapi import FastAPI, status, Body

api = FastAPI()

# -- To Check connection
@app.get("/")
def read_root(): # function that is binded with the endpoint
    return {"Welcome to the API"}
```


To use the database we have to connect to it,
for MongoDB we need a `connection_string`,
which I will call `URL_DB` and because we are using the localhost we will set it as defined in the previous section `URL_DB = mongodb://localhost:27017/` and the database name will be the same as before `testDB`.
We will store the variable `URL_DB` inside an `.env` file.

To work with the DB we will have to get the MongoDB Client,
for that we will add the following lines to the file:

```python
import os
from motor.motor_asyncio import AsyncIOMotorClient
from fastapi import FastAPI, status, Body

api = FastAPI()

# -- To Check connection
@app.get("/")
def read_root(): # function that is binded with the endpoint
    return {"Welcome to the API"}

# -- Connect to DB
# DB_URL = "mongodb://localhost:27017/"
client = AsyncIOMotorClient(os.environ[DB_URL])
# -- Get database
DB = client["testDB"]
```

We will add an endpoint to creat a User and post it into the DB.
To do that we first need to define two data-models to handle conversion and manipulation of data between the API and the DB,
one is the `PyObjectId` and the other one is the `UserModel`.

For `pydantic < 2.0.0` we will use:

```python
from bson import ObjectId
from pydantic import BaseModel, Field
from typing import Optional

class PyObjectId(ObjectId):
    @classmethod
    def __get_validators__(cls):
        yield cls.validate

    @classmethod
    def validate(cls, v):
        if not ObjectId.is_valid(v):
            raise ValueError("Invalid objectid")
        return ObjectId(v)

    # NOTE: invalid for pydantic-v2
    @classmethod
    def __modify_schema__(cls, field_schema):
        field_schema.update(type="string")

class UserModel(BaseModel):
    id: PyObjectId = Field(default_factory=PyObjectId, alias="_id")
    email: str = Field(...)
    password: str = Field(...)
    name: Optional[str] = Field(...)

    class Config:
        allow_population_by_field_name = True
        arbitrary_types_allowed = True
        json_encoders = {ObjectId: str}
        schema_extra = {
            "example": {
                "email":"demo@mail.com",
                "password": "demo_secret_pass",
                "name":"Demo Account"
            }
        } 
```

For `pydantic >= 2.0.0` we can replace `PyObjectId` by:
```python
from typing_extensions import Annotated
from pydantic.functional_validators import BeforeValidator
PyObjectId = Annotated[str, BeforeValidator(str)]
```

Add the endpoint to create a new user:
```python
...
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse

...
@api.post(
    "/users/",
    response_description="Add new user",
    response_model=UserModel,
)
async def create_user(user: UserModel):
    # TODO: validate user doesn't already exist.
    user_json = jsonable_encoder(user)
    new_user = await DB["users"].insert_one(user_json)
    created_user = await DB["users"].find_one(
        {"_id": new_user.inserted_id}
    )
    res = JSONResponse(
        status_code=status.HTTP_201_CREATED, 
        content=created_user
    )
    return res
```

Add an endpoint to list all existing Users:
```python
from typing import List

...

@api.get(
    "/users",
    response_description="List all users",
    response_model=List[UserModel]
)
async def list_users():
    users = await DB["users"].find().to_list(1000)
    return users
```

Lets organize the source code into it's own dir:

```shell
mkdir src 
```

Inside we will create a few files:
-   `__init__.py` to create a pkg
-   `models.py` to hold all Type Models to interact between the DB and the API. 
-   `routers.py` to hold all endpoints.
-   `settings.py` for database connection


1.  Move the `UserModel` and `PyObjectId` to `src/models`.
2.  Move endpoints ƒrom `main.py` to `router` and import `routers` to `main` with `api.include_router(router)`.

## 4 - Connect API to MongoDB database

>[!NOTE]
> MongoDB should be running with the Docker container

As is we can connect but we won't be able to interact yet.

To connect we have to run the API in the server,
in this case a local one using [Uvicorn]().
Since we are managing the project with UV we will run it inside its `venv`.
Make sure you are inside `api` when running the following:

In [ ]:
uv run uvicorn main:api --reload

## 5 - Replace the manual setup with Docker Compose

We will create a container for the API, 
another for the Database, 
with a volume to persist the data,
and then connect them via a network.

Create a file `docker-compose.yml` and inside it set the following:
```yaml
version: "3.8"
services:
```


We need to create the container for MongoDB database and initialize the database before we connect it to the API.
For the MongoDB database we will create a service `database` that will use the latest version of `mongo`
(a better practice is to specify a version)

```yaml
version: "3.8"
services:
    database:
        image: mongo
```

Since we want the data we store in the DB to persist after container up/down operations,
we introduce a volume that we call `mongodata` and we map it to the database container.

```yaml
version: "3.8"
services:
    database:
        image: mongo
        volumes:
            - mongodata:/data/db
volumes:
    mongodata:
```

Next we have to create the Server Side API container.
For that we will need to create inside `api/` a `Dockerfile` and a `requirements.txt` file.
The latter contains all the dependencies required to build and install the API and run it in the server.

Example of the `requirements.txt` file:

```
asgiref==3.4.1
bcrypt==3.2.0
cffi==1.14.5
click==8.0.1
colorama==0.4.4
cryptography==3.4.7
dnspython==2.1.0
ecdsa==0.17.0
email-validator==1.1.3
fastapi==0.66.0
h11==0.12.0
idna==3.2
motor==2.4.0
passlib==1.7.4
pyasn1==0.4.8
pycparser==2.20
pydantic==1.8.2
pymongo==3.11.4
python-jose==3.3.0
python-multipart==0.0.5
rsa==4.7.2
six==1.16.0
starlette==0.14.2
typing-extensions==3.10.0.0
uvicorn==0.14.0
```

One can extract one from UV (it uses `uv.lock`) using:
```bash
uv export --no-hashes --format requirements-txt > requirements.txt
```
Or using pip:
```bash
pip3 freeze > requirements.txt 
```

Create the `Dockerfile` for the API.
We will base the API in the v3.8 of python,
we will declare the `WORKDIR` inside the container,
then we will copy the `requirements.txt` file into the working directory and install all dependencies.
Then we copy the source code into the working directory,
and we `EXPOSE` a port to listen to the request in the server.
Finally we run the API in the server using the `CMD`. 

The file will look like this:
```dockerfile
# Pull image
FROM python:3.8-slim-buster

# setup workdir same as said in docker compose
WORKDIR /code

#   install inside container
COPY requirements.txt requirements.txt
RUN pip3 install -r requirements.txt

#   port that FastAPI will be listening on inside the container
EXPOSE 8000
COPY . .

CMD ["uvicorn", "main:api", "--host", "0.0.0.0", "--port", "8000"]
```

Now we have to add that container "recipe" to our docker compose file.
For that we first need to create a service `api` that will use the above created `Dockerfile`.
```yaml
version: "3.8"
services:
    api:
        build: ./api
        volumes:
            - ./api/:/code
        ports:
            - 8000:8000
    database:
        image: mongo
        volumes:
            - mongodata:/data/db
volumes:
    mongodata:
```
We add the flag `build: ./api` to specify that the source code dir is the one that locally is called `./api`.
We add a volume to store the source code inside the containers `WORKDIR` as expected by the `Dockerfile`.
Then we connected the local-machine port `8000` with the exposed container port `8000`
(again as expected and defined in the `Dockerfile`).
Note that the machine port doesn't need to be the same as the container port.

So far we have all the containers that we need,
but they are not connected and thus the API will not work as it will fail to interact with the database.
Docker Compose will handle creating the network,
we only need to specify a few parameters.
Since the MongoDB client uses a connection string that we called `DB_URL` we have to set it here or pass an `.env` file with that variable.
Note that I'm using the name of the service to connect the API w/ the DB.
We will also state that we want the API to wait for the database service before starting.
In addition I will expose a port to access the DB from MongoDB Compass using `port 27017`.

The file becomes:
```yaml
version: "3.8"
services:
    api:
        build: ./api
        volumes:
            - ./api/:/code
        ports:
            - 8000:8000
        environment:
            - DB_URL=mongodb://database/testDB
        depends_on:
            - database
    database:
        image: mongo
        volumes:
            - mongodata:/data/db
        ports:
            - 27017:27017
volumes:
    mongodata:
```

-----